In [4]:
# !pip install rdflib
#% !pip install morph-kgc

In [5]:
import morph_kgc
g = morph_kgc.materialize('../mappings/config.ini')

g.serialize(destination='resultado.nt', format='nt')
print("Archivo resultado.nt generado")

INFO | 2025-11-07 13:14:19,159 | Parallelization is not supported for win32 when running as a library. If you need to speed up your data integration pipeline, please run through the command line.
INFO | 2025-11-07 13:14:22,562 | 80 mapping rules retrieved.
INFO | 2025-11-07 13:14:22,662 | Mapping partition with 80 groups generated.
INFO | 2025-11-07 13:14:22,664 | Maximum number of rules within mapping group: 1.
INFO | 2025-11-07 13:14:22,665 | Mappings processed in 3.497 seconds.
INFO | 2025-11-07 13:14:24,409 | Number of triples generated in total: 7968.
WARNING | 2025-11-07 13:14:24,432 | Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#decimal, Converter=<class 'decimal.Decimal'>
Traceback (most recent call last):
  File "c:\Users\julia\AppData\Local\Programs\Python\Python312\Lib\site-packages\rdflib\term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
decimal.Inv

Archivo resultado.nt generado


In [6]:
from rdflib import Graph
g = Graph()

In [7]:
g.parse("../ontology/ontology.ttl", format="turtle")
g.parse("resultado.nt", format="nt")

WARNING | 2025-11-07 13:14:25,833 | Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#decimal, Converter=<class 'decimal.Decimal'>
Traceback (most recent call last):
  File "c:\Users\julia\AppData\Local\Programs\Python\Python312\Lib\site-packages\rdflib\term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
decimal.InvalidOperation: [<class 'decimal.ConversionSyntax'>]
WARNING | 2025-11-07 13:14:25,836 | Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#decimal, Converter=<class 'decimal.Decimal'>
Traceback (most recent call last):
  File "c:\Users\julia\AppData\Local\Programs\Python\Python312\Lib\site-packages\rdflib\term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
decimal.InvalidOperation: [<class 'decimal.ConversionSyntax'>]
WARNING | 2025-11-07 1

<Graph identifier=N8ed8173cb51d4fe7837164e252a19a9a (<class 'rdflib.graph.Graph'>)>

In [8]:
PREFIXES = """
PREFIX sc:     <http://smartcity.linkeddata.es/lcc/ontology#> 
PREFIX schema: <https://schema.org/> 
PREFIX geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#> 
PREFIX dbo:    <http://dbpedia.org/ontology/> 
PREFIX owl:    <http://www.w3.org/2002/07/owl#> 
PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX xsd:    <http://www.w3.org/2001/XMLSchema#> 
"""


print(f"Triples cargados: {len(g)}")

# Mostrar las primeras 10 tripletas
for i, (s, p, o) in enumerate(g):
    print(s, p, o)
    if i >= 25:
        break

Triples cargados: 8109
http://smartcity.linkeddata.es/lcc/resource/facility/park/Pinar-Santa-Eugenia http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://schema.org/ParkGarden
http://smartcity.linkeddata.es/lcc/resource/subway/Metro%3A%20%20Atocha https://schema.org/name Metro:  Atocha
http://smartcity.linkeddata.es/lcc/ontology#hasStations http://www.w3.org/2000/01/rdf-schema#domain http://smartcity.linkeddata.es/lcc/ontology#Train
http://smartcity.linkeddata.es/lcc/resource/facility/library/Biblioteca-Publica-Municipal-Huerta-de-la-Salud-%28Hortaleza%29 https://schema.org/name Biblioteca Pública Municipal Huerta de la Salud (Hortaleza)
http://smartcity.linkeddata.es/lcc/resource/neighbourhood/ARGUELLES https://schema.org/name ARGUELLES
http://smartcity.linkeddata.es/lcc/resource/facility/sportCenter/Centro-Deportivo-Municipal-Maria-Jesus-Rosa-%28La-Vaguada%29 http://www.w3.org/2003/01/geo/wgs84_pos#long -3.7086993281311904
http://smartcity.linkeddata.es/lcc/resource/facility/park/

In [9]:
print(f"Triples cargados: {len(g)}")

PREFIXES = """
PREFIX sc:     <http://smartcity.linkeddata.es/lcc/ontology#>
PREFIX schema: <https://schema.org/>
PREFIX dbo:    <http://dbpedia.org/ontology/>
PREFIX geo:    <http://www.w3.org/2003/01/geo/wgs84_pos#>
"""

def qrun(q, limit=50):
    q = PREFIXES + q
    res = g.query(q)
    print(f"--- {len(res)} filas ---")
    for i,row in enumerate(res):
        print([str(x) if x is not None else None for x in row])
        if i >= limit-1: break

Triples cargados: 8109


In [10]:
# ✅ Número de individuos de cada subclase de Facility (tabla con tipo y conteo)
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a sc:EducationalInstitution   BIND("EducationalInstitution"    AS ?type) } UNION
  { ?x a schema:Library      BIND("PublicLibrary" AS ?type) } UNION
  { ?x a schema:ParkGarden   BIND("ParkGarden"    AS ?type) } UNION
  { ?x a sc:SportsCenter BIND("SportsCenter"  AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")


--- 4 filas ---
['EducationalInstitution', '118']
['ParkGarden', '203']
['PublicLibrary', '50']
['SportsCenter', '81']


In [11]:
# 🔎 5 ejemplos de cada subclase (solo nombre)
# - Lista 5 nombres por subclase (si falta schema:name, usa el localname del IRI)

for subtype,label in [
    ("sc:EducationalInstitution","EducationalInstitutions"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters")
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?x a {subtype} .
  OPTIONAL {{ ?x schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?x), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### EducationalInstitutions
--- 5 filas ---
['Universidad Complutense de Madrid. Facultad de Medicina']
['Colegio Mayor Diego de Covarrubias']
['Colegio Mayor Argentino Nuestra Señora de Luján']
['Colegio Mayor Santa María del Pino']
['Universidad Autónoma de Madrid. Facultad de Ciencias Económicas y Empresariales']

### PublicLibraries
--- 5 filas ---
['Biblioteca Pública Miguel Hernández (Puente de Vallecas)']
['Biblioteca Pública Municipal Pablo Neruda (Ciudad Lineal)']
['Biblioteca Pública Municipal La Elipa (Ciudad Lineal)']
['Biblioteca Pública Hortaleza']
['Biblioteca Pública José Hierro (Usera)']

### ParksGardens
--- 5 filas ---
['Pinar Santa Eugenia']
['Parque Este de Valdebernardo']
['Área Forestal de Tres Cantos']
['Jardines de Gloria Fuertes']
['Parque Ciudad de los Ángeles']

### SportsCenters
--- 5 filas ---
['Centro Deportivo Municipal Gallur']
['Centro Deportivo Municipal San Fermín']
['Centro Deportivo Municipal Peñagrande']
['Centro Deportivo Municipal José María Ca

In [12]:
# 🚇 Número de individuos de Metro, Bus y Tren (Cercanías)
# - Tabla con tipo de transporte y conteo

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a schema:SubwayStation        BIND("Subway"        AS ?type) } UNION
  { ?x a schema:BusStation           BIND("Bus"           AS ?type) } UNION
  { ?x a schema:TrainStation BIND("SuburbanTrain" AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")


--- 3 filas ---
['Bus', '254']
['SuburbanTrain', '50']
['Subway', '263']


In [13]:
# 🚏 5 ejemplos de Metro, Bus y Tren (solo nombre)
# - Muestra 5 nombres por tipo de transporte

for ttype,label in [
    ("schema:SubwayStation","Subway"),
    ("schema:BusStation","Bus"),
    ("schema:TrainStation","SuburbanTrain")
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?t a {ttype} .
  OPTIONAL {{ ?t schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?t), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### Subway
--- 5 filas ---
['Metro: El Capricho (línea 5)']
['Metro: Carabanchel Alto (línea 11)']
['Metro: Moncloa (línea 3 y 6) , Plaza de España (línea 2, 3 y 10) y Príncipe Pío (línea 6 y 10)']
['Metro: Lago, Puerta del Ángel']
['Metro: Iglesia (línea 1)']

### Bus
--- 5 filas ---
['Bus%3A%208%2C%2019%2C%2045%2C%2047%2C%2059%2C%2085%2C%2086%20y%20247']
['Bus%3A%20102']
['Bus%3A%20%2054%2C%20103%2C%20142']
['Bus%3A%203%2C%2060%20y%20148']
['Bus%3A%20121%20%2C%20131%20%2C%20139']

### SuburbanTrain
--- 5 filas ---
['Renfe: Embajadores (línea C5)']
['Renfe: Chamartín C1, C2, C3, C4a, C4b, C7 y C8']
['Renfe: Delicias c1 c10']
['Vicálvaro C2']
['Renfe: Las Águilas C5']


In [14]:
# 🗺️ Número de individuos de Municipio, Distrito y Barrio
# - Tabla con tipo territorial y conteo

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a dbo:Municipality  BIND("Municipality"  AS ?type) } UNION
  { ?x a dbo:District      BIND("District"      AS ?type) } UNION
  { ?x a dbo:Neighbourhood BIND("Neighbourhood" AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")


--- 3 filas ---
['District', '22']
['Municipality', '1']
['Neighbourhood', '118']


In [15]:
# 🏷️ 5 ejemplos de Municipio, Distrito y Barrio (solo nombre)
# - Muestra 5 nombres por cada clase territorial

for ctype,label in [
    ("dbo:Municipality", "Municipality"),
    ("dbo:District", "District"),
    ("dbo:Neighbourhood", "Neighbourhood"),
]:
    print(f"\n### {label}")
    qrun(f"""
SELECT ?name WHERE {{
  ?x a {ctype} .
  OPTIONAL {{ ?x schema:name ?raw }}
  BIND(COALESCE(?raw, REPLACE(STR(?x), '^.*/', '')) AS ?name)
}}
LIMIT 5
""", limit=5)



### Municipality
--- 1 filas ---
['MADRID']

### District
--- 5 filas ---
['MONCLOA-ARAVACA']
['CHAMBERI']
['DISTRITO']
['CHAMARTIN']
['CIUDAD LINEAL']

### Neighbourhood
--- 5 filas ---
['JUSTICIA/ CENTRO']
['VALVERDE']
['COMILLAS']
['ARCOS']
['PALOS DE MOGUER']


In [16]:
# 🔁 Comprobar jerarquía Municipio→Distrito→Barrio
# - Lista pares Municipio-Distrito y Distrito-Barrio (nombres) para verificar la estructura

print("### Neigbourhood → District")
qrun("""
SELECT ?municipality ?district WHERE {
  ?m a dbo:Neighbourhood ; schema:name ?municipality ; sc:locatedInDistrict ?d .
  OPTIONAL { ?d schema:name ?district }
}
ORDER BY ?municipality ?district
""", limit=20)

print("\n### District → Municipality")
qrun("""
SELECT ?district ?neighbourhood WHERE {
  ?d a dbo:District ; schema:name ?district ; sc:locatedInMunicipality ?n .
  OPTIONAL { ?n schema:name ?neighbourhood }
}
ORDER BY ?district ?neighbourhood
""", limit=20)


### Neigbourhood → District
--- 119 filas ---
['ABRANTES', 'CARABANCHEL']
['ACACIAS', 'ARGANZUELA']
['ADELFAS', 'RETIRO']
['ALAMEDA DE OSUNA', 'BARAJAS']
['ALMAGRO', 'CHAMBERI']
['ALMENARA', 'TETUAN']
['ALMENDRALES', 'USERA']
['ALUCHE', 'LATINA']
['AMBROZ', 'VICALVARO']
['APOSTOL SANTIAGO', 'HORTALEZA']
['ARAPILES', 'CHAMBERI']
['ARAVACA', 'MONCLOA-ARAVACA']
['ARCOS', 'SAN BLAS-CANILLEJAS']
['ARGUELLES', 'MONCLOA-ARAVACA']
['ATALAYA', 'CIUDAD LINEAL']
['BELLAS VISTAS', 'TETUAN']
['BERRUGUETE', 'TETUAN']
['BUENAVISTA', 'CARABANCHEL']
['BUTARQUE', 'VILLAVERDE']
['CANILLAS', 'HORTALEZA']

### District → Municipality
--- 22 filas ---
['ARGANZUELA', 'MADRID']
['BARAJAS', 'MADRID']
['CARABANCHEL', 'MADRID']
['CENTRO', 'MADRID']
['CHAMARTIN', 'MADRID']
['CHAMBERI', 'MADRID']
['CIUDAD LINEAL', 'MADRID']
['DISTRITO', 'MADRID']
['FUENCARRAL-EL PARDO', 'MADRID']
['HORTALEZA', 'MADRID']
['LATINA', 'MADRID']
['MONCLOA-ARAVACA', 'MADRID']
['MORATALAZ', 'MADRID']
['PUENTE DE VALLECAS', 'MADRID']
['RE

In [17]:
# 🧭 Comprobar relaciones subclases de Facility → Barrio
# - Para cada subclase: lista 5 (Facility, Barrio) por containedInPlace

for subtype,label in [
    ("sc:EducationalInstitution","Universities"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters")
]:
    print(f"\n### {label} → Neighbourhood")
    qrun(f"""
SELECT ?facility ?neighbourhood WHERE {{
  ?f a {subtype} ; schema:containedInPlace ?n .
  ?n a dbo:Neighbourhood .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?n schema:name ?nraw }}
  BIND(COALESCE(?fraw, REPLACE(STR(?f), '^.*/', '')) AS ?facility)
  BIND(COALESCE(?nraw, REPLACE(STR(?n), '^.*/', '')) AS ?neighbourhood)
}}
LIMIT 5
""", limit=5)



### Universities → Neighbourhood
--- 5 filas ---
['Universidad Complutense de Madrid. Facultad de Medicina', 'CIUDAD UNIVERSITARIA']
['Colegio Mayor Diego de Covarrubias', 'CIUDAD UNIVERSITARIA']
['Colegio Mayor Argentino Nuestra Señora de Luján', 'CIUDAD UNIVERSITARIA']
['Colegio Mayor Santa María del Pino', 'VALLEHERMOSO']
['Universidad Autónoma de Madrid. Facultad de Ciencias Económicas y Empresariales', 'EL GOLOSO']

### PublicLibraries → Neighbourhood
--- 5 filas ---
['Biblioteca Pública Miguel Hernández (Puente de Vallecas)', 'PALOMERAS SURESTE']
['Biblioteca Pública Municipal Pablo Neruda (Ciudad Lineal)', 'PUEBLO NUEVO']
['Biblioteca Pública Municipal La Elipa (Ciudad Lineal)', 'VENTAS']
['Biblioteca Pública Hortaleza', 'PINAR DEL REY']
['Biblioteca Pública José Hierro (Usera)', 'PRADOLONGO']

### ParksGardens → Neighbourhood
--- 5 filas ---
['Pinar Santa Eugenia', 'CASCO H.VICALVARO']
['Parque Este de Valdebernardo', 'CASCO H.VICALVARO']
['Área Forestal de Tres Cantos', 'EL P

In [18]:
# 🚈 Comprobar relaciones subclases de Facility → Metro/Bus/Tren
# - Para cada subclase: cuenta cuántas instalaciones tienen cada tipo de transporte cercano

for subtype,label in [
    ("sc:EducationalInstitution","Universities"),
    ("schema:Library","PublicLibraries"),
    ("schema:ParkGarden","ParksGardens"),
    ("sc:SportsCenter","SportsCenters")
]:
    
    print(f"\n### {label} con transporte cercano (conteos por tipo)")
    qrun(f"""
SELECT ?transportType (COUNT(DISTINCT ?f) AS ?n) WHERE {{
  ?f a {subtype} ; sc:hasNearby ?t .
  {{
    ?t a schema:SubwayStation        BIND("Subway" AS ?transportType)
  }} UNION {{
    ?t a schema:BusStation           BIND("Bus" AS ?transportType)
  }} UNION {{
    ?t a schema:TrainStation BIND("SuburbanTrain" AS ?transportType)
  }}
}}
GROUP BY ?transportType
ORDER BY DESC(?n)
""", limit=10)

    print(f"### {label} → ejemplos (nombre facility, tipo transporte, nombre transporte)")
    qrun(f"""
SELECT ?facility ?tType ?tName WHERE {{
  ?f a {subtype} ; sc:hasNearby ?t .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?t schema:name ?tname }}
  BIND(COALESCE(?fraw, REPLACE(STR(?f), '^.*/', '')) AS ?facility)
  BIND(COALESCE(?tname, REPLACE(STR(?t), '^.*/', '')) AS ?tName)
  {{
    ?t a schema:SubwayStation        BIND("Subway" AS ?tType)
  }} UNION {{
    ?t a schema:BusStation           BIND("Bus" AS ?tType)
  }} UNION {{
    ?t a schema:TrainStation BIND("SuburbanTrain" AS ?tType)
  }}
}}
LIMIT 5
""", limit=5)



### Universities con transporte cercano (conteos por tipo)
--- 3 filas ---
['Subway', '99']
['SuburbanTrain', '24']
['Bus', '2']
### Universities → ejemplos (nombre facility, tipo transporte, nombre transporte)
--- 5 filas ---
['Universidad Complutense de Madrid. Facultad de Medicina', 'Subway', 'Metro: Ciudad Universitaria (línea 6)']
['Colegio Mayor Diego de Covarrubias', 'Subway', 'Metro: Moncloa (Línea 6), Ciudad Universitaria (Línea 6)']
['Colegio Mayor Argentino Nuestra Señora de Luján', 'Subway', 'Metro: Moncloa (línea 6) Ciudad Universitaria (línea 6)']
['Colegio Mayor Santa María del Pino', 'Subway', 'Metro: Islas Filipinas (línea 7)']
['Universidad Autónoma de Madrid. Facultad de Ciencias Económicas y Empresariales', 'SuburbanTrain', 'Cantoblanco Universidad C4']

### PublicLibraries con transporte cercano (conteos por tipo)
--- 3 filas ---
['Subway', '45']
['SuburbanTrain', '13']
['Bus', '1']
### PublicLibraries → ejemplos (nombre facility, tipo transporte, nombre transport

In [19]:
# ✅ Número de individuos de cada subclase de Facility (tabla con tipo y conteo)
# - Devuelve una fila por subclase: University, Library, ParkGarden, SportsCenter

qrun("""
SELECT ?type (COUNT(?x) AS ?count) WHERE {
  { ?x a sc:Facility   BIND("University"    AS ?type) }
}
GROUP BY ?type
ORDER BY ?type
""")

--- 1 filas ---
['University', '452']


In [20]:
# --------------------------------------------------------------------
query_2 = """
SELECT (COUNT(?park) AS ?total)  # Cambiamos ?count a ?total para evitar colisiones con el método
WHERE {
?park a schema:ParkGarden ;
        sc:accessibility "Instalación parcialmente accesible para personas con movilidad reducida" .
}
"""

print("\n--- Resultados de la Consulta 2: Conteo de Parques Parcialmente Accesibles ---")
try:
    results_2 = g.query(PREFIXES + query_2)
    
    # COUNT devuelve una única fila. Iteramos sobre ella y accedemos al valor.
    for row in results_2:

        count_value = row.total.value if hasattr(row.total, 'value') else row.total
        print(f"Total de Parques Parcialmente Accesibles: {count_value}")

        
except Exception as e:
    print(f"Error al ejecutar la Consulta 2: {e}")


--- Resultados de la Consulta 2: Conteo de Parques Parcialmente Accesibles ---
Total de Parques Parcialmente Accesibles: 197


In [21]:
# 🧭 Comprobar relaciones subclases de Facility → Barrio
# - Para cada subclase: lista 5 (Facility, Barrio) por containedInPlace

for subtype,label in [
    ("sc:Facility","Facilities")
]:
    print(f"\n### {label} → Neighbourhood")
    qrun(f"""
SELECT ?facility ?neighbourhood WHERE {{
  ?f a {subtype} ; schema:containedInPlace ?n .
  ?n a dbo:Neighbourhood .
  OPTIONAL {{ ?f schema:name ?fraw }}
  OPTIONAL {{ ?n schema:name ?nraw }}
  BIND(COALESCE(?fraw, REPLACE(STR(?f), '^.*/', '')) AS ?facility)
  BIND(COALESCE(?nraw, REPLACE(STR(?n), '^.*/', '')) AS ?neighbourhood)
}}
LIMIT 5
""", limit=5)


### Facilities → Neighbourhood
--- 5 filas ---
['Centro Deportivo Municipal Orcasur', 'ORCASUR']
['Jardín del Bulevar de Peña Gorbea', 'SAN DIEGO']
['Universidad Complutense de Madrid. Facultad de Derecho', 'CIUDAD UNIVERSITARIA']
['Escuela Técnica Superior de Ingeniería y Diseño Industrial', 'PALOS DE LA FRONTERA']
['Parque Payaso Fofó', 'PALOMERAS BAJAS']
